In [2]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
import tensorflow as tf
#from train import create_dir, load_dataset

In [3]:
global image_h
global image_w
global num_landmarks

In [9]:
train_x = sorted(glob(os.path.join(path, "train", "images", "*.jpg")))
train_y = sorted(glob(os.path.join(path, "train", "landmarks", "*.txt")))

valid_x = sorted(glob(os.path.join(path, "val", "images", "*.jpg")))
valid_y = sorted(glob(os.path.join(path, "val", "landmarks", "*.txt")))

test_x = sorted(glob(os.path.join(path, "test", "images", "*.jpg")))
test_y = sorted(glob(os.path.join(path, "test", "landmarks", "*.txt")))

In [60]:
def plot_landmarks(image, landmarks):
    h, w, _ = image.shape
    radius = int(h * 0.005)

    for i in range(0, len(landmarks), 2):
        x = int(landmarks[i] * w)
        y = int(landmarks[i+1] * h)

        image = cv2.circle(image, (x, y), radius, (255, 0, 0), -1)

    return image

In [8]:
""" Seeding """
np.random.seed(42)
tf.random.set_seed(42)

""" Directory for storing files """
# create_dir("results")

""" Hyperparameters """
image_h = 512
image_w = 512
num_landmarks = 106

""" Paths """
path = "./LaPa"
model_path = os.path.join("files_test", "model.h5")

In [11]:
 """ Loading the dataset """
# (train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_dataset(path)
print(f"Train: {len(train_x)}/{len(train_y)} - Valid: {len(valid_x)}/{len(valid_y)} - Test: {len(test_x)}/{len(test_x)}")
print("")


Train: 18168/18168 - Valid: 2000/2000 - Test: 2000/2000



In [12]:
""" Load the model """
model = tf.keras.models.load_model(model_path)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 256, 256, 16  432         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 256, 256, 16  64          ['Conv1[0][0]']                  
                                )                                                             

                                                                                                  
 block_3_expand (Conv2D)        (None, 128, 128, 96  1536        ['block_2_add[0][0]']            
                                )                                                                 
                                                                                                  
 block_3_expand_BN (BatchNormal  (None, 128, 128, 96  384        ['block_3_expand[0][0]']         
 ization)                       )                                                                 
                                                                                                  
 block_3_expand_relu (ReLU)     (None, 128, 128, 96  0           ['block_3_expand_BN[0][0]']      
                                )                                                                 
                                                                                                  
 block_3_p

                                                                                                  
 block_6_depthwise_BN (BatchNor  (None, 32, 32, 96)  384         ['block_6_depthwise[0][0]']      
 malization)                                                                                      
                                                                                                  
 block_6_depthwise_relu (ReLU)  (None, 32, 32, 96)   0           ['block_6_depthwise_BN[0][0]']   
                                                                                                  
 block_6_project (Conv2D)       (None, 32, 32, 32)   3072        ['block_6_depthwise_relu[0][0]'] 
                                                                                                  
 block_6_project_BN (BatchNorma  (None, 32, 32, 32)  128         ['block_6_project[0][0]']        
 lization)                                                                                        
          

 lization)                                                                                        
                                                                                                  
 block_10_expand_relu (ReLU)    (None, 32, 32, 192)  0           ['block_10_expand_BN[0][0]']     
                                                                                                  
 block_10_depthwise (DepthwiseC  (None, 32, 32, 192)  1728       ['block_10_expand_relu[0][0]']   
 onv2D)                                                                                           
                                                                                                  
 block_10_depthwise_BN (BatchNo  (None, 32, 32, 192)  768        ['block_10_depthwise[0][0]']     
 rmalization)                                                                                     
                                                                                                  
 block_10_

 alization)                                                                                       
                                                                                                  
 block_14_expand (Conv2D)       (None, 16, 16, 480)  38400       ['block_13_project_BN[0][0]']    
                                                                                                  
 block_14_expand_BN (BatchNorma  (None, 16, 16, 480)  1920       ['block_14_expand[0][0]']        
 lization)                                                                                        
                                                                                                  
 block_14_expand_relu (ReLU)    (None, 16, 16, 480)  0           ['block_14_expand_BN[0][0]']     
                                                                                                  
 block_14_depthwise (DepthwiseC  (None, 16, 16, 480)  4320       ['block_14_expand_relu[0][0]']   
 onv2D)   

 dense (Dense)                  (None, 212)          271572      ['dropout[0][0]']                
                                                                                                  
Total params: 977,796
Trainable params: 959,252
Non-trainable params: 18,544
__________________________________________________________________________________________________


In [15]:
""" Prediction """
for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):
    """ Extract the name """
    name = x.split("/")[-1].split(".")[0]

    """ Reading the image """
    image = cv2.imread(x, cv2.IMREAD_COLOR)
    image_x = image
    image = cv2.resize(image, (image_w, image_h))
    image = image/255.0 ## (512, 512, 3)
    image = np.expand_dims(image, axis=0) ## (1, 512, 512, 3)
    image = image.astype(np.float32)

    """ Landmarks """
    data = open(y, "r").read()
    landmarks = []
    for line in data.strip().split("\n")[1:]:
        x, y = line.split(" ")
        x = float(x)/image_x.shape[1]
        y = float(y)/image_x.shape[0]

        landmarks.append(x)
        landmarks.append(y)

    landmarks = np.array(landmarks, dtype=np.float32)

    """ Prediction """
    pred = model.predict(image, verbose=0)[0]
    pred = pred.astype(np.float32)

    """ Saving the results """
    gt_landmarks = plot_lankmarks(image_x.copy(), landmarks)
    pred_landmarks = plot_lankmarks(image_x.copy(), pred)
    line = np.ones((image_x.shape[0], 10, 3)) * 255

    cat_images = np.concatenate([gt_landmarks, line, pred_landmarks], axis=1)
    cv2.imwrite(f"results/{name}.png", cat_images)

100%|██████████████████████████████████████████████████████████████████████████████| 2000/2000 [13:30<00:00,  2.47it/s]


# test only one image

In [63]:
x = test_x
y = test_y
x

['./LaPa\\test\\images\\10001868414_0.jpg',
 './LaPa\\test\\images\\10009865324_0.jpg',
 './LaPa\\test\\images\\10012551673_5.jpg',
 './LaPa\\test\\images\\10012551673_8.jpg',
 './LaPa\\test\\images\\10014368575_1.jpg',
 './LaPa\\test\\images\\10014368575_2.jpg',
 './LaPa\\test\\images\\10014990385_0.jpg',
 './LaPa\\test\\images\\10014990385_1.jpg',
 './LaPa\\test\\images\\10020518914_0.jpg',
 './LaPa\\test\\images\\10022989606_0.jpg',
 './LaPa\\test\\images\\10023065833_0.jpg',
 './LaPa\\test\\images\\10023946615_0.jpg',
 './LaPa\\test\\images\\10027676166_3.jpg',
 './LaPa\\test\\images\\100301707_2.jpg',
 './LaPa\\test\\images\\10036490423_1.jpg',
 './LaPa\\test\\images\\10036529425_0.jpg',
 './LaPa\\test\\images\\10037648863_0.jpg',
 './LaPa\\test\\images\\10041043896_2.jpg',
 './LaPa\\test\\images\\10041890394_0.jpg',
 './LaPa\\test\\images\\10044526923_0.jpg',
 './LaPa\\test\\images\\10044601603_0.jpg',
 './LaPa\\test\\images\\10049319155_0.jpg',
 './LaPa\\test\\images\\1005034959

In [64]:
x = x[1]
y = y[1]
x

'./LaPa\\test\\images\\10009865324_0.jpg'

In [65]:
# name = x.split("/")[-1].split(".")[0]
name = x.split("/")[1].split('\\')[3].split(".")[0]
name

'10009865324_0'

In [66]:
""" Reading the image """
image = cv2.imread(x, cv2.IMREAD_COLOR)
image_x = image
image = cv2.resize(image, (image_w, image_h))
image = image/255.0 ## (512, 512, 3)
image = np.expand_dims(image, axis=0) ## (1, 512, 512, 3)
image = image.astype(np.float32)
image

array([[[[0.73333335, 0.9529412 , 0.972549  ],
         [0.7254902 , 0.9490196 , 0.96862745],
         [0.7137255 , 0.9490196 , 0.9647059 ],
         ...,
         [0.47058824, 0.49803922, 0.47058824],
         [0.47058824, 0.49803922, 0.47058824],
         [0.47843137, 0.5019608 , 0.47843137]],

        [[0.7372549 , 0.95686275, 0.9764706 ],
         [0.7294118 , 0.9529412 , 0.972549  ],
         [0.7137255 , 0.9490196 , 0.96862745],
         ...,
         [0.47058824, 0.49803922, 0.47058824],
         [0.47058824, 0.49803922, 0.47058824],
         [0.4745098 , 0.5019608 , 0.4745098 ]],

        [[0.7411765 , 0.9607843 , 0.98039216],
         [0.7294118 , 0.95686275, 0.972549  ],
         [0.7176471 , 0.9529412 , 0.96862745],
         ...,
         [0.4745098 , 0.5058824 , 0.4745098 ],
         [0.4745098 , 0.5019608 , 0.4745098 ],
         [0.47843137, 0.5058824 , 0.47843137]],

        ...,

        [[0.08627451, 0.08627451, 0.08627451],
         [0.07843138, 0.07843138, 0.07843138]

In [67]:
data = open(y, "r").read()
landmarks = []

In [68]:
data.strip().split("\n")[1:]

['179.000000 254.000000',
 '177.000000 274.000000',
 '177.000000 294.000000',
 '177.000000 314.000000',
 '178.000000 334.000000',
 '181.000000 354.000000',
 '185.000000 374.000000',
 '191.000000 394.000000',
 '200.000000 412.000000',
 '212.000000 428.000000',
 '227.000000 442.000000',
 '243.000000 454.000000',
 '261.000000 464.000000',
 '281.000000 473.000000',
 '302.000000 479.000000',
 '324.000000 482.000000',
 '346.000000 482.000000',
 '365.000000 479.000000',
 '381.000000 472.000000',
 '395.000000 462.000000',
 '407.000000 451.000000',
 '418.000000 439.000000',
 '427.000000 425.000000',
 '435.000000 410.000000',
 '441.000000 394.000000',
 '445.000000 377.000000',
 '448.000000 361.000000',
 '451.000000 344.000000',
 '454.000000 327.000000',
 '455.000000 310.000000',
 '456.000000 294.000000',
 '456.000000 277.000000',
 '455.000000 260.000000',
 '233.000000 228.000000',
 '248.000000 213.000000',
 '269.000000 208.000000',
 '292.000000 212.000000',
 '310.000000 221.000000',
 '307.000000

In [69]:
for line in data.strip().split("\n")[1:]:
    x, y = line.split(" ")
    x = float(x)/image_x.shape[1]
    y = float(y)/image_x.shape[0]

    landmarks.append(x)
    landmarks.append(y)
landmarks = np.array(landmarks, dtype=np.float32)

In [70]:
'661.000000 1064.000000'.split(" ")

['661.000000', '1064.000000']

In [71]:
landmarks

array([0.2745399 , 0.41100323, 0.2714724 , 0.4433657 , 0.2714724 ,
       0.47572815, 0.2714724 , 0.5080906 , 0.27300614, 0.5404531 ,
       0.27760735, 0.57281554, 0.28374234, 0.605178  , 0.2929448 ,
       0.63754046, 0.30674848, 0.6666667 , 0.32515338, 0.6925566 ,
       0.34815952, 0.7152104 , 0.37269938, 0.73462784, 0.40030676,
       0.7508091 , 0.4309816 , 0.76537216, 0.4631902 , 0.7750809 ,
       0.4969325 , 0.7799353 , 0.5306749 , 0.7799353 , 0.55981594,
       0.7750809 , 0.58435583, 0.76375407, 0.6058282 , 0.74757284,
       0.6242331 , 0.72977346, 0.6411043 , 0.710356  , 0.654908  ,
       0.68770224, 0.6671779 , 0.6634304 , 0.6763804 , 0.63754046,
       0.6825153 , 0.6100324 , 0.68711656, 0.5841424 , 0.6917178 ,
       0.5566343 , 0.69631904, 0.5291262 , 0.6978528 , 0.50161815,
       0.6993865 , 0.47572815, 0.6993865 , 0.44822007, 0.6978528 ,
       0.42071196, 0.35736197, 0.36893204, 0.38036808, 0.3446602 ,
       0.41257668, 0.33656958, 0.44785276, 0.34304208, 0.47546

In [72]:
""" Prediction """
pred = model.predict(image, verbose=0)[0]
pred = pred.astype(np.float32)
pred

array([0.23968536, 0.40041426, 0.24209411, 0.43040857, 0.23327139,
       0.4738402 , 0.2276475 , 0.5039926 , 0.22797234, 0.5413581 ,
       0.23450568, 0.5817953 , 0.23280802, 0.61165637, 0.2377507 ,
       0.64963216, 0.24538466, 0.68736625, 0.2566956 , 0.72178125,
       0.28554103, 0.75781375, 0.30506817, 0.78022945, 0.33032307,
       0.79935056, 0.35847855, 0.82266885, 0.3972068 , 0.8372223 ,
       0.43898258, 0.84758884, 0.47736123, 0.8533501 , 0.5126543 ,
       0.8560739 , 0.5443713 , 0.84787333, 0.56996584, 0.8300994 ,
       0.6037154 , 0.8112916 , 0.62543046, 0.79450136, 0.64823157,
       0.77335334, 0.66840005, 0.74711335, 0.6909297 , 0.71669567,
       0.6973438 , 0.6944659 , 0.7127726 , 0.66203165, 0.71690863,
       0.6344887 , 0.72514975, 0.5979883 , 0.7300478 , 0.5656164 ,
       0.7428565 , 0.529915  , 0.7381235 , 0.509232  , 0.7480744 ,
       0.46204668, 0.32456374, 0.39159486, 0.37003347, 0.36878386,
       0.41501278, 0.35920817, 0.45425102, 0.37881103, 0.50314

In [73]:
""" Saving the results """
gt_landmarks = plot_landmarks(image_x.copy(), landmarks)
pred_landmarks = plot_landmarks(image_x.copy(), pred)
line = np.ones((image_x.shape[0], 10, 3)) * 255


In [74]:
cat_images = np.concatenate([gt_landmarks, line, pred_landmarks], axis=1)
cv2.imwrite(f"./results/{name}.png", cat_images)

True